In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Clone YOLOv5 and install dependencies
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt  # install dependencies

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")


Cloning into 'yolov5'...
remote: Enumerating objects: 16953, done.
remote: Counting objects: 100% (148/148), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 16953 (delta 75), reused 98 (delta 48), pack-reused 16805 (from 1)
Receiving objects: 100% (16953/16953), 15.71 MiB | 17.24 MiB/s, done.
Resolving deltas: 100% (11608/11608), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 109.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 871.8/871.8 kB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.4 MB/s eta 0:00:00
Setup complete. Using torch 2.4.0+cu121 (NVIDIA L4)


In [ ]:
# Install Roboflow
!pip install roboflow

# Download the first dataset from the first Roboflow account
from roboflow import Roboflow
rf1 = Roboflow(api_key="c9Z5KFzF8r7yAKNHNYPl")
project1 = rf1.workspace("mdp3").project("mdp-3-sjijk")
version1 = project1.version(2)
dataset1 = version1.download("yolov5")

# Download the second dataset from the second Roboflow account
from roboflow import Roboflow
rf2 = Roboflow(api_key="RsMYxVSAo6lZ66IfMhLe")
project2 = rf2.workspace("image-recognition-mdp").project("mdp2-n2ksj")
version2 = project2.version(4)
dataset2 = version2.download("yolov5")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.8
    Uninstalling idna-3.8:
      Successfully uninstalled idna-3.8
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to MDP-3-2 in yolov5pytorch:: 100%|██████████| 56751/56751 [00:09<00:00, 5899.68it/s]


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to MDP2-4 in yolov5pytorch:: 100%|██████████| 51026/51026 [00:07<00:00, 6589.79it/s]


In [ ]:
import shutil
import os

# Paths to your datasets
dataset1_path = '/content/yolov5/MDP-3-2/'
dataset2_path = '/content/yolov5/MDP2-4/'
merged_dataset_path = '/content/yolov5/merged_dataset/'

# Create the merged dataset directories
for folder in ['train', 'valid', 'test']:
    os.makedirs(merged_dataset_path + f'{folder}/images', exist_ok=True)
    os.makedirs(merged_dataset_path + f'{folder}/labels', exist_ok=True)

# Function to merge the contents of images and labels folders
def merge_folders(src, dst):
    for folder_name in ['train', 'valid', 'test']:
        images_src = os.path.join(src, folder_name, 'images')
        labels_src = os.path.join(src, folder_name, 'labels')
        images_dst = os.path.join(dst, folder_name, 'images')
        labels_dst = os.path.join(dst, folder_name, 'labels')

        # Copy images
        for image_file in os.listdir(images_src):
            shutil.copy(os.path.join(images_src, image_file), images_dst)

        # Copy labels
        for label_file in os.listdir(labels_src):
            shutil.copy(os.path.join(labels_src, label_file), labels_dst)

# Merge dataset 1 (MDP-3-2)
merge_folders(dataset1_path, merged_dataset_path)

# Merge dataset 2 (MDP-2-4)
merge_folders(dataset2_path, merged_dataset_path)

print("Datasets merged successfully!")


Datasets merged successfully!


In [ ]:
# Display GPU info
!nvidia-smi

Wed Sep 11 01:31:58 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA L4                      Off | 00000000:00:03.0 Off |                    0 |
| N/A   51C    P8              18W /  72W |      4MiB / 23034MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
# Create the necessary directory first
os.makedirs('/content/yolov5/data/hyps', exist_ok=True)

# Create hyp.scratch.yaml file with basic hyperparameters
# Add lrf to the existing hyp.scratch.yaml file
hyp_content = """
# YOLOv5 hyperparameters
lr0: 0.001  # initial learning rate (SGD=1E-2, Adam=1E-3)
lrf: 0.2  # final learning rate (fraction of lr0)
momentum: 0.937  # SGD momentum/Adam beta1
weight_decay: 0.0005  # optimizer weight decay
warmup_epochs: 3.0  # warmup epochs (fractions ok)
warmup_momentum: 0.8  # warmup initial momentum
warmup_bias_lr: 0.1  # warmup initial bias lr
box: 0.05  # box loss gain
cls: 0.5  # cls loss gain
cls_pw: 1.0  # cls BCELoss positive_weight
obj: 1.0  # obj loss gain
obj_pw: 1.0  # obj BCELoss positive_weight
iou_t: 0.20  # IoU training threshold
anchor_t: 4.0  # anchor-multiple threshold
fl_gamma: 0.0  # focal loss gamma (efficientDet default is gamma=1.5)
hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
hsv_v: 0.4  # image HSV-Value augmentation (fraction)
degrees: 0.0  # image rotation (+/- deg)
translate: 0.1  # image translation (+/- fraction)
scale: 0.5  # image scale (+/- gain)
shear: 0.0  # image shear (+/- deg)
perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
flipud: 0.0  # image flip up-down (probability)
fliplr: 0.5  # image flip left-right (probability)
mosaic: 1.0  # image mosaic (probability)
mixup: 0.0  # image mixup (probability)
copy_paste: 0.0  # segment copy-paste (probability)
"""
with open('/content/yolov5/data/hyps/hyp.scratch.yaml', 'w') as f:
    f.write(hyp_content)

# Verify the file was created
!ls /content/yolov5/data/hyps/hyp.scratch.yaml


/content/yolov5/data/hyps/hyp.scratch.yaml


In [ ]:
# Path to the new data.yaml file
yaml_file_path = "/content/yolov5/merged_dataset/data.yaml"

# Content of the new data.yaml file
data_yaml_content = """
train: /content/yolov5/merged_dataset/train/images
val: /content/yolov5/merged_dataset/valid/images
test: /content/yolov5/merged_dataset/test/images  # Optional if you're using a test set

nc: 31

names:
- A
- B
- Bullseye
- C
- D
- E
- F
- G
- H
- S
- T
- U
- V
- W
- X
- Y
- Z
- circle
- down
- eight
- five
- four
- left
- nine
- one
- right
- seven
- six
- three
- two
- up
"""

# Write the data.yaml content to the specified file
with open(yaml_file_path, "w") as f:
    f.write(data_yaml_content)

print(f"data.yaml file created at {yaml_file_path}")


data.yaml file created at /content/yolov5/merged_dataset/data.yaml


In [ ]:
#!python /content/yolov5/train.py \
  --img 416 \
  --batch 16 \
  --epochs 200 \
  --data /content/yolov5/merged_dataset/data.yaml \
  --weights /content/drive/MyDrive/yolov5_training/experiment1/weights/best.pt \
  --hyp /content/yolov5/data/hyps/hyp.scratch.yaml \
  --project /content/drive/MyDrive/yolov5_training/experiment2 \
  --name weights

Streaming output truncated to the last 5000 lines.
  with torch.cuda.amp.autocast(amp):
     44/199      1.81G   0.009305   0.001908   0.001364         23        416:  91% 2944/3220 [04:06<00:21, 12.61it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
     44/199      1.81G   0.009304   0.001908   0.001364         27        416:  91% 2944/3220 [04:06<00:21, 12.61it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
     44/199      1.81G   0.009304   0.001908   0.001365         36        416:  91% 2946/3220 [04:06<00:22, 12.21it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.aut

In [ ]:
!python /content/yolov5/train.py --resume /content/drive/MyDrive/yolov5_training/experiment2/weights/weights/last.pt


Streaming output truncated to the last 5000 lines.
  with torch.cuda.amp.autocast(amp):
    199/199       1.8G   0.006092   0.001387  0.0005189         22        416:  23% 742/3220 [01:01<03:17, 12.56it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
    199/199       1.8G   0.006089   0.001387  0.0005184         21        416:  23% 744/3220 [01:01<03:11, 12.94it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
    199/199       1.8G     0.0061   0.001388  0.0005181         23        416:  23% 744/3220 [01:02<03:11, 12.94it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autoca

In [ ]:
!python /content/yolov5/detect.py \
  --weights /content/drive/MyDrive/yolov5_training/experiment2/weights/weights/best.pt \
  --img 416 \
  --conf 0.25 \
  --source /content/yolov5/merged_dataset/test/images \
  --save-txt \
  --save-conf \
  --project /content/drive/MyDrive/yolov5_training/experiment2/detection_results \
  --name test_results


detect: weights=['/content/drive/MyDrive/yolov5_training/experiment2/weights/weights/best.pt'], source=/content/yolov5/merged_dataset/test/images, data=data/coco128.yaml, imgsz=[416, 416], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_format=0, save_csv=False, save_conf=True, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=/content/drive/MyDrive/yolov5_training/experiment2/detection_results, name=test_results, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-365-g12b577c8 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)

Fusing layers... 
Model summary: 157 layers, 7093732 parameters, 0 gradients, 16.0 GFLOPs
WARNING ⚠️ NMS time limit 0.550s exceeded
image 1/818 /content/yolov5/merged_dataset/test/images/0133_jpg.rf.8f8f7cfaec1bf5b8c37d025b4a9151e9.jpg: 416x416 1 C, 6.6ms
image 2/818 /con

In [ ]:
!python /content/yolov5/val.py \
  --weights /content/drive/MyDrive/yolov5_training/experiment2/weights/weights/best.pt \
  --img 416 \
  --data /content/yolov5/merged_dataset/data.yaml \
  --iou 0.65 \
  --conf 0.25


val: data=/content/yolov5/merged_dataset/data.yaml, weights=['/content/drive/MyDrive/yolov5_training/experiment2/weights/weights/best.pt'], batch_size=32, imgsz=416, conf_thres=0.25, iou_thres=0.65, max_det=300, task=val, device=, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
WARNING ⚠️ confidence threshold 0.25 > 0.001 produces invalid results
YOLOv5 🚀 v7.0-365-g12b577c8 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (NVIDIA L4, 22700MiB)

Fusing layers... 
Model summary: 157 layers, 7093732 parameters, 0 gradients, 16.0 GFLOPs
val: Scanning /content/yolov5/merged_dataset/valid/labels.cache... 1542 images, 0 backgrounds, 0 corrupt: 100% 1542/1542 [00:00<?, ?it/s]
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 49/49 [00:11<00:00,  4.18it/s]
                   all       1542       1556      0.981     